In [1]:
import numpy as np

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.1/389.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.0/112.0 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    

In [5]:
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader

from transformers import AdamW
from tqdm import tqdm

/Users/seein/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import pandas as pd

In [7]:
# KcELECTRA 모델과 토크나이저 불러오기
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)


/Users/seein/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
data = pd.read_excel('한국어_단발성_대화_데이터셋.xlsx')
data

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [10]:
# 라벨링
data.loc[(data['Emotion'] == "공포"), 'Emotion'] = 0       #공포=> 1
data.loc[(data['Emotion'] == "놀람"), 'Emotion'] = 2       #분노 => 0
data.loc[(data['Emotion'] == "분노"), 'Emotion'] = 1       #놀람 => 2
data.loc[(data['Emotion'] == "슬픔"), 'Emotion'] = 3       #슬픔 => 3
data.loc[(data['Emotion'] == "중립"), 'Emotion'] = 4       #중립 => 4
data.loc[(data['Emotion'] == "행복"), 'Emotion'] = 5       #행복 => 5
data.loc[(data['Emotion'] == "혐오"), 'Emotion'] = 6       #혐오 => 6


data_list = []
for ques, label in zip(data['Sentence'], data['Emotion']):
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [11]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=2000)

In [12]:
print(len(dataset_train), len(dataset_test))

28945 9649


In [13]:
dataset_train

[['일본도 상위선수 평균연봉이20~30억 수준인데 개거품 이네', '1'],
 ['서민들은 피눈물이 납니다', '1'],
 ['이런 내일부터 일하는데 바들바들 떨준비해야긋넹....다들 감기조심하세요!!', '0'],
 ['뭐지 이 무리수 제목은?', '2'],
 ['그냥 허망하다는 말밖에 안나옵니다..', '0'],
 ['순실이 뒷통수 거하게 맞았노 ㅋㅋㅋㅋㅋ', '5'],
 [' 저희 엄마도 동생을 못보게 하네요.', '3'],
 ['소녀상을 세워서 일제의 만행을 잊지않도록 해야만 합니다.', '4'],
 ['갤노트 버리고 V20사게? ㅋㅋㅋㅋ', '4'],
 ['이햐 공유 돈좀 벌겠는데요', '3'],
 [' 기말을 1주컷으로 끝내야되네 ㅠ0', '3'],
 ['    김연아님 진심으로 축하합니다. ^^', '5'],
 ['보면 5~6000 천이 우습게 나오던데 뭐하시는분들이신지 ㄷㄷㄷ', '2'],
 ['사람들 기대치가 점점 더 커져서 내가 다 긴장됨 ㅋ', '0'],
 ['부산행 영화 보는거 같다~~~~소오름~~~', '2'],
 ['파업하는 현대차 사지 맙시다.', '6'],
 ['취임후 나라들을 뒤집을 정책을 펼치고 있으면서 곳곳에서 시위하는데 호화휴가라ㅋㅋ', '3'],
 ['참여해주시면 감사합니당 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', '5'],
 ['저한테 지쳐서 헤어졌고 저는 엄청잡았었어요', '3'],
 ['이 빌어쳐먹을 매국노년', '6'],
 ['이름그대로 쥐변호사가 딱어울리네', '1'],
 ['전국민 앞에서 손에다 장만 지지고 아무것도 하지마라!!!', '6'],
 ['조의연 판사한테는 믿고 싶지 않은 뉴스겠군요....', '4'],
 ['이런말은 24일 합숙하고도 3일 훈련한 팀한테 진 감독이 할 말은 아닌 것 같은데', '6'],
 ['조리장은 매일밤 오이에 콘돔 씌워서 가져다 드려라....자위대놀이하시게...', '1'],
 ['정준영 반갑더라..역시 차태현 ㅋ', '5'],
 ['여

In [14]:
# 여기서부터

In [15]:
# 데이터셋 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence, label = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        label = torch.tensor(int(label), dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

In [16]:
# 학습 데이터셋과 테스트 데이터셋 준비
train_dataset = EmotionDataset(dataset_train, tokenizer)
test_dataset = EmotionDataset(dataset_test, tokenizer)

In [17]:
train_dataset[0]

{'input_ids': tensor([    2, 15201, 13525, 11699, 12960, 21504,  4034, 10604,    96, 25048,
         24860, 15621, 15603,     3,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [18]:
# DataLoader 정의 (배치 단위로 데이터를 불러오기 위함)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [19]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [20]:
# 옵티마이저 정의
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프 정의
epochs = 7


/Users/seein/miniconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        # 데이터 준비
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델에 입력 데이터 전달 후 출력 획득
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}")

  0%|          | 2/453 [00:43<2:43:33, 21.76s/it]


KeyboardInterrupt: 

In [15]:
model.eval()  # 평가 모드로 설정
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy}")

100%|██████████| 604/604 [01:18<00:00,  7.69it/s]

Test Accuracy: 0.5752927764535185


In [16]:
def analyze_diary(model, tokenizer, diary_text):
    model.eval()
    encoding = tokenizer.encode_plus(
        diary_text,
        add_special_tokens=True,
        max_length=128,
        return_attention_mask=True,
        return_tensors='pt',
        padding='max_length',
        truncation=True
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    emotions = ["공포", "분노", "놀람", "슬픔", "중립", "행복", "혐오"]
    intensity = probabilities.squeeze().cpu().numpy()

    for i, emotion in enumerate(emotions):
        print(f"{emotion}: {intensity[i]:.2f}")

In [19]:
# 사용자가 입력한 일기 예시
diary_text = "행복하지만 힘들기도 하고 좋기도 하고 슬프기도 해"
analyze_diary(model, tokenizer, diary_text)

공포: 0.00
분노: 0.00
놀람: 0.00
슬픔: 1.00
중립: 0.00
행복: 0.00
혐오: 0.00
